In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
from fredapi import Fred

In [3]:
fred = Fred(api_key = '')

# Federal Reserve Economic Data Overview:

The purpose of this notebook is to showcase how to retrieve economic data from the Federal Reserve Ecomomic Data API and how to visualize the data into interative graphs. 

In this notebook, the following several economic indicators will be extracted
- Bond Yields
- GDP
- Household Debt
- Disposible Income
- Unemployment Rate
- Housing Starts

The data will also be converted into YoY percentages as typical of economic data.

*In order to access the FRED API, an account must be created for an API Key* <br>
https://research.stlouisfed.org/docs/api/api_key.html

# Retrieving Economic Reports from FRED API

In order to retrieve reports the report ID number must be identified and can be located near the title of the reports

In [4]:
bonds = fred.get_series('DGS10')
gdp = fred.get_series('A191RL1Q225SBEA')
houshold_debt = fred.get_series('TDSP')
income = fred.get_series('DSPI')
unemployment = fred.get_series('UNRATE')
housing_starts = fred.get_series('HOUST')

In [5]:
type(bonds)

pandas.core.series.Series

Note that the data type is a Series

### Manipulates series data into Data Frames and Renames the Columns

In [6]:
bonds = fred.get_series('DGS10')
bonds = bonds.to_frame().reset_index()
bonds.rename(columns = {'index': 'Date', 0:'Interest Rate'}, inplace = True)

gdp = fred.get_series('A191RL1Q225SBEA')
gdp = gdp.to_frame().reset_index()
gdp.rename(columns = {'index': 'Date', 0:'GDP Percentage'}, inplace = True)

household_debt = fred.get_series('TDSP')
household_debt = household_debt.to_frame().reset_index()
household_debt.rename(columns = {'index': 'Date', 0:'Household Debt'}, inplace = True)

income = fred.get_series('DSPI')
income = income.to_frame().reset_index()
income.rename(columns = {'index': 'Date', 0:'Disposible Income'}, inplace = True)

unemployment = fred.get_series('UNRATE')
unemployment = unemployment.to_frame().reset_index()
unemployment.rename(columns = {'index': 'Date', 0:'Unemployment Rate'}, inplace = True)

housing_starts = fred.get_series('HOUST')
housing_starts = housing_starts.to_frame().reset_index()
housing_starts.rename(columns = {'index': 'Date', 0:'Housing Starts'}, inplace = True)

In [7]:
bonds.head()

,Date,Interest Rate
0,1962-01-02,4.06
1,1962-01-03,4.03
2,1962-01-04,3.99
3,1962-01-05,4.02
4,1962-01-08,4.03


In [8]:
bonds.dtypes

Date             datetime64[ns]
Interest Rate           float64
dtype: object

Note that the data type for Date is in datetime allowing us to use any datetime functions

### Filter each dataset from 2017

In [9]:
bonds = bonds[bonds['Date']  >= '2017-01-01']
gdp = gdp[gdp['Date']  >= '2017-01-01']
household_debt = household_debt[household_debt['Date']  >= '2017-01-01']
income = income[income['Date'] >= '2017-01-01']
unemployment = unemployment[unemployment['Date'] >= '2017-01-01']
housing_starts = housing_starts[housing_starts['Date'] >= '2017-01-01']
bonds.head()

,Date,Interest Rate
14349,2017-01-02,NaN
14350,2017-01-03,2.45
14351,2017-01-04,2.46
14352,2017-01-05,2.37
14353,2017-01-06,2.42


In [10]:
gdp.head()

,Date,GDP Percentage
279,2017-01-01,1.8
280,2017-04-01,3.0
281,2017-07-01,2.8
282,2017-10-01,2.3
283,2018-01-01,2.2


In [11]:
housing_starts.head()

,Date,Housing Starts
696,2017-01-01,1225.0
697,2017-02-01,1289.0
698,2017-03-01,1179.0
699,2017-04-01,1165.0
700,2017-05-01,1122.0


**Note** <br>
It's important to note that economic datasets do not come in the same cadence or frequency. For example Bond Yields are on a daily level, while GDP Data is on a quarterly level, while Housing Starts data comes in every month.

This is important to distinguish especially when building any models based off economic data.

# Building Interactive Graphs using Plotly

Plotly is a library in python where one can create more iteractive graphs compared to the static graphs typically made through matplotlib.
<br>
There is an online mode where one would register for free, but for the purpose of this exercise the offline mode will be implemented

In [12]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets

#used for plotly offline mode
py.offline.init_notebook_mode(connected = True)

In [13]:
layout = go.Layout(
    title ='Bond Yields',
    yaxis = dict(
    title = 'Interest Rates'),
    xaxis = dict(
    title = 'Day'),
)


trace1 = go.Scatter(
    x = bonds['Date'],
    y = bonds['Interest Rate'],
    mode = 'lines',
    name = 'Daily Interest Rates',
    line = dict(
    shape = 'spline')

)

fig = go.Figure(data = [trace1], layout = layout)
py.offline.iplot(fig)

### Creating 20 Day Moving Average and Adding into Graph

In [14]:
bonds['MA'] = bonds['Interest Rate'].rolling(window = 30).mean()
bonds.tail()

,Date,Interest Rate,MA
14878,2019-01-11,2.71,NaN
14879,2019-01-14,2.71,NaN
14880,2019-01-15,2.72,NaN
14881,2019-01-16,2.73,NaN
14882,2019-01-17,2.75,NaN


In [15]:
layout = go.Layout(
    title ='Bond Yields',
    yaxis = dict(
    title = 'Interest Rates'),
    xaxis = dict(
    title = 'Day'),
)


trace1 = go.Scatter(
    x = bonds['Date'],
    y = bonds['Interest Rate'],
    mode = 'lines',
    name = 'Daily Interest Rates',
    line = dict(
    shape = 'spline')

)


trace2 = go.Scatter(
    x = bonds['Date'],
    y = bonds['MA'],
    mode = 'lines',
    name = '20 Day Moving Average',
    line = dict(
    shape = 'spline')

)

fig = go.Figure(data = [trace1,trace2], layout = layout)
py.offline.iplot(fig)

Calculating the 20 Day Moving Average and vizualizing it on the graph shows that the function is not working entirely as expected and this is due to the gaps in the data which can be seen whenever there are gaps in the Blue Line (Daily Interest Rates)

# Summary
Economic Data can be pulled via API as showcased with the Federal Reserve Economic Data (FRED) and be be visualized using libraries such as plotly in python. <br>

Things that should be taken note of when extracing economic data are as followed:
* The cadence of the economic data will vary depending on the type of information being reported and should be taken note of or accounted for when building for any modeling purposes. 
<br>
* Data may be incomplete, which will affect functions such as the moving average

### How to Improve
Data can be cleaned up for the moving average to work properly. For example, missing values can be replaced with average values, or the moving average function can be revised to handle missing days.

YoY% percentages can be calculated using the python function pct_change

The notebook can be further expanded using other Economic Data Sources such as Bureau of Economic Analysis (BEA), US Census Bureau, and IMF Data